In [27]:
import requests
from bs4 import BeautifulSoup
import time
from lxml import html
import sqlite3

first_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13104&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1"
base_url = "https://suumo.jp/"
page_num = 0

try:
    r = requests.get(first_url)
except requests.exceptions.RequestException as e:
    print(e)
except requests.exceptions.ConnectionError as e:
    print(e)
except requests.exceptions.Timeout as e:
    print(e)
except requests.exceptions.HTTPError as e:
    print(e)
except requests.exceptions.TooManyRedirects as e:
    print(e)
soup = BeautifulSoup(r.text, "html.parser")
page = soup.find_all("ol", class_="pagination-parts")
page_list = page[0].find_all("li")
page_num_max = page_list[-1].find("a").text

In [28]:
# sqlite3のインポート
conn = sqlite3.connect('suumo.db')
c = conn.cursor()

# テーブルの作成
c.execute('''CREATE TABLE IF NOT EXISTS suumo(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            property TEXT,
            property_title TEXT,
            location TEXT,
            rent INTEGER,
            Management_fee INTEGER,
            deposit INTEGER,
            gratuity INTEGER,
            Security_deposit INTEGER,
            expense INTEGER,
            floor_plan TEXT,
            buildings_age TEXT,
            floor TEXT,
            Building_type TEXT,
            url TEXT)''')

In [29]:
def Duplication_check():
    c.execute("SELECT url FROM suumo")
    url_list = c.fetchall()
    return url_list

In [30]:
def insert_data(property, property_title, location, rent, Management_fee, deposit, gratuity, Security_deposit, expense, floor_plan, buildings_age, floor, Building_type, url):
    conn = sqlite3.connect('suumo.db')
    c = conn.cursor()
    # データの重複チェックを行い、重複していない場合はデータを挿入
    c.execute("SELECT url FROM suumo WHERE url = ?", (url,))
    url_list = c.fetchall()
    if len(url_list) == 0:
        c.execute("INSERT INTO suumo(property, property_title, location, rent, Management_fee, deposit, gratuity, Security_deposit, expense, floor_plan, buildings_age, floor, Building_type, url) VALUES(? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,?)", (property, property_title, location, rent, Management_fee, deposit, gratuity, Security_deposit, expense, floor_plan, buildings_age, floor, Building_type, url))
        conn.commit()
        return True
    else:
        print("重複データのためスキップ")
        return False

In [31]:
def isint(i):
    try:
        int(i)
        return True
    except ValueError:
        return False

def isfloat(i):
    try:
        float(i)
    except ValueError:
        return False
    else:
        return True

In [32]:
def scrape_data(ul_list):
    index_num = 0
    for ul in ul_list:
        div_list = ul.find_all("div", class_="cassetteitem")
        for div in div_list:
            title = div.find("div", class_="cassetteitem_content-title").text
            print("=================================================================")
            print(title)
            tr_list = div.find_all("tr", class_="js-cassette_link")
            for tb in tr_list:
                cas_tbody = tb.find("a", class_="js-cassette_link_href cassetteitem_other-linktext").get("href")
                url = base_url.rstrip('/') + cas_tbody
                # 取得したURLを使って、次のページに遷移して情報を取得する
                try:
                    r = requests.get(url)
                except requests.exceptions.RequestException as e:
                    print(e)
                    continue
                except requests.exceptions.ConnectionError as e:
                    print(e)
                    continue
                except requests.exceptions.Timeout as e:
                    print(e)
                    continue
                except requests.exceptions.HTTPError as e:
                    print(e)
                    continue
                except requests.exceptions.TooManyRedirects as e:
                    print(e)
                    continue
                soup = BeautifulSoup(r.text, "html.parser")
                xml = html.fromstring(str(soup))
                time.sleep(0.5)
                try:
                    property_title = soup.find("h1", class_="section_h1-header-title").text
                except AttributeError:
                    property_title = None
                try:
                    base_rent = soup.find("span", class_="property_view_note-emphasis").text
                    rent = base_rent.replace("万円", "")
                    if isfloat(rent):
                        rent = int(float(rent) * 10000)
                    else:
                        rent = None
                except AttributeError:
                    rent = None
                location = soup.select('#js-view_gallery > div.l-property_view_table > table > tr:nth-child(1) > td')
                try:
                    location = location[0].get_text()
                except IndexError:
                    location = None
                Management_fee = soup.select("#js-view_gallery > div.property_view_note > div.property_view_note-info > div:nth-child(1) > span:nth-child(2)")
                try:
                    Management_fee = Management_fee[0].get_text()
                    Management_fee = Management_fee.replace("管理費・共益費:", "")
                    Management_fee = Management_fee.replace("円", "")
                    Management_fee = Management_fee.replace("-", "None")
                    Management_fee = Management_fee.lstrip()
                except IndexError:
                    Management_fee = None
                deposit = soup.select("#js-view_gallery > div.property_view_note > div.property_view_note-info > div:nth-child(2) > span:nth-child(1)")
                try:
                    deposit = deposit[0].get_text()
                    deposit = deposit.replace("敷金:", "")
                    deposit = deposit.replace("万円", "")
                    deposit = deposit.replace("-", "None")
                    deposit = deposit.lstrip()
                    if isfloat(deposit):
                        deposit = int(float(deposit) * 10000)
                    else:
                        deposit = None
                except IndexError:
                    deposit = None
                gratuity = soup.select("#js-view_gallery > div.property_view_note > div.property_view_note-info > div:nth-child(2) > span:nth-child(2)")
                try:
                    gratuity = gratuity[0].get_text()
                    gratuity = gratuity.replace("礼金:", "")
                    gratuity = gratuity.replace("万円", "")
                    gratuity = gratuity.replace("-", "None")
                    gratuity = gratuity.lstrip()
                    if isfloat(gratuity):
                        gratuity = int(float(gratuity) * 10000)
                    else:
                        gratuity = None
                except IndexError:
                    gratuity = None
                Security_deposit = soup.select("#js-view_gallery > div.property_view_note > div.property_view_note-info > div:nth-child(2) > span:nth-child(3)")
                try:
                    Security_deposit = Security_deposit[0].get_text()
                    Security_deposit = Security_deposit.replace("保証金:", "")
                    Security_deposit = Security_deposit.replace("万円", "")
                    Security_deposit = Security_deposit.replace("-", "None")
                    Security_deposit = Security_deposit.lstrip()
                    if Security_deposit == isfloat(Security_deposit):
                        Security_deposit = int(float(Security_deposit) * 10000)
                    else:
                        Security_deposit = None
                except IndexError:
                    Security_deposit = None
                expense = soup.select("#js-view_gallery > div.property_view_note > div.property_view_note-info > div:nth-child(2) > span:nth-child(4)")
                try:
                    expense = expense[0].get_text()
                    expense = expense.replace("敷引・償却:", "")
                    expense = expense.replace("万円", "")
                    expense = expense.replace("-", "None")
                    expense = expense.lstrip()
                    if isfloat(expense):
                        expense = int(float(expense) * 10000)
                    else:
                        expense = None
                except IndexError:
                    expense = None
                floor_plan = soup.select("#js-view_gallery > div.l-property_view_table > table > tr:nth-child(3) > td:nth-child(2)")
                try:
                    floor_plan = floor_plan[0].get_text()
                except IndexError:
                    floor_plan = None
                buildings_age = soup.select('#js-view_gallery > div.l-property_view_table > table > tr:nth-child(4) > td:nth-child(2)')
                try:
                    buildings_age = buildings_age[0].get_text()
                    if buildings_age == "新築":
                        buildings_age = 0
                    else:
                        buildings_age = buildings_age.replace("築", "")
                        buildings_age = buildings_age.replace("年", "")
                        buildings_age = buildings_age.replace("以上", "")
                        buildings_age = int(buildings_age)
                except IndexError:
                    buildings_age = None
                floor = soup.select("#js-view_gallery > div.l-property_view_table > table > tr:nth-child(4) > td:nth-child(4)")
                try:
                    floor = floor[0].get_text()
                except IndexError:
                    floor = None
                Building_type = soup.select("#js-view_gallery > div.l-property_view_table > table > tr:nth-child(5) > td:nth-child(4)")
                try:
                    Building_type = Building_type[0].get_text()
                except IndexError:
                    Building_type = None
                print("-----------------------------------------------------------------")
                print('データベースの数 : ' + str(index_num))
                # データベースに格納
                if insert_data(title, property_title, location, rent, Management_fee, deposit, gratuity, Security_deposit, expense, floor_plan, buildings_age, floor, Building_type, url):
                    index_num += 1
                



In [33]:
for i in range(page_num, int(page_num_max)):
    url = first_url + "&page=" + str(i)
    print(url)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    time.sleep(1)
    ul_list = soup.find_all("ul", class_="l-cassetteitem")
    scrape_data(ul_list)

conn.close()

https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13104&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page=0
ＪＲ山手線 高田馬場駅 3階建 築7年
-----------------------------------------------------------------
データベースの数 : 1
重複データのためスキップ
都営大江戸線 牛込神楽坂駅 4階建 築2年
-----------------------------------------------------------------
データベースの数 : 1
重複データのためスキップ
ザ・パークハウス西新宿タワー60
-----------------------------------------------------------------
データベースの数 : 1
重複データのためスキップ
-----------------------------------------------------------------
データベースの数 : 1
重複データのためスキップ
-----------------------------------------------------------------
データベースの数 : 1
重複データのためスキップ
-----------------------------------------------------------------
データベースの数 : 1
重複データのためスキップ
-----------------------------------------------------------------
データベースの数 : 1
重複データのためスキップ
-----------------------------------------------------------------
データベースの数 : 1
重複データのためスキップ
------------

KeyboardInterrupt: 